<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>

# LangGraph Router Pattern: Intent-Based Agent Routing
In this tutorial, we'll explore the Router pattern using LangGraph, a framework for building dynamic, stateful LLM applications. We’ll build a smart support assistant that can route customer queries to the appropriate specialized agent—Billing, Tech Support, or General Info—based on the user’s intent.

LangGraph enables this by letting us define a structured graph of logic, where a router node classifies the input, and conditional edges forward it to the correct sub-agent. Each agent uses local context (e.g., invoice history, troubleshooting tips) to respond intelligently.

We also trace this application using Phoenix, which gives us complete visibility into routing decisions, tool usage, and model interactions. This is helpful for debugging routing accuracy and understanding how the graph executes end-to-end.

In [ ]:
!pip install langgraph langchain langchain_community "arize-phoenix" arize-phoenix-otel openinference-instrumentation-langchain

In [ ]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.6/437.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.56
    Uninstalling langchain-core-0.3.56:
      Successfully uninstalled langchain-core-0.3.56


In [ ]:
from langgraph.graph import StateGraph, START, END
import os, getpass

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


# Configure Phoenix Tracing

Make sure you go to https://app.phoenix.arize.com/ and generate an API key. This will allow you to trace your Langgraph application with Phoenix.

In [ ]:
PHOENIX_API_KEY = getpass.getpass("Phoenix API Key:")
os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"

Phoenix API Key:··········


In [ ]:
from phoenix.otel import register

tracer_provider = register(
  project_name="Router",
  auto_instrument=True
)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: Router
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://app.phoenix.arize.com/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {'api_key': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [ ]:
from typing_extensions import TypedDict, Literal
from IPython.display import Image, display

from langchain_openai import ChatOpenAI

# LLM

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

In [ ]:
from typing import Literal, TypedDict
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import SystemMessage, HumanMessage


/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


# Router
We define a Pydantic schema to structure the router’s output. This schema ensures the LLM returns one of three valid routing targets: "billing", "tech_support", or "general_info".
We then wrap our LLM with with_structured_output, allowing LangGraph to enforce this structured response during routing.

In [ ]:
class Route(BaseModel):
    step: Literal["billing", "tech_support", "general_info"] = Field(description="Classify the support request")

router = llm.with_structured_output(Route)


/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1630: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


# Defining Graph State
This state schema captures the lifecycle of a routed request. It stores the original user input (input), the classification decision made by the router (decision), and the final response generated by the appropriate support handler (output). Each node in the graph will read from and write to this shared state.

In [ ]:
class State(TypedDict):
    input: str
    decision: str
    output: str

# Support Agent Nodes: Specialized Response Handlers
This section defines three specialized LLM-powered agents, each responsible for handling a different category of user queries:

**Billing Agent**: Uses user billing history and a billing policy context to respond to invoice or refund-related questions.

**Tech Support Agent**: Answers common troubleshooting queries using a predefined support knowledge base.

**General Info Agent**: Responds to account, subscription, and policy-related questions using general FAQs.

Each agent reads the input field from the graph's state and returns a generated output tailored to its domain. These agents form the execution endpoints of the router graph.

In [ ]:
# Billing Agent
billing_history_data = [
    {"invoice_id": "INV001", "date": "2024-11-01", "amount": "$29.99"},
    {"invoice_id": "INV002", "date": "2024-12-01", "amount": "$29.99"},
    {"invoice_id": "INV003", "date": "2025-01-01", "amount": "$39.99"},
]

billing_general_context = (
    "Billing inquiries may include refunds, invoices, plan upgrades, or charges. "
    "Our system charges users monthly based on their plan. Refunds are processed within 5–7 business days."
)

def billing_agent(state: State):
    user_input = state["input"]
    invoice_summary = "\n".join(
        f"• Invoice {item['invoice_id']} on {item['date']} for {item['amount']}"
        for item in billing_history_data
    )
    prompt = (
        f"You are a helpful billing assistant.\n"
        f"Here is the user's billing history:\n{invoice_summary}\n\n"
        f"General billing context:\n{billing_general_context}\n\n"
        f"User query:\n{user_input}"
    )
    result = llm.invoke(prompt)
    return {"output": result.content}


# Tech Support Agent
tech_support_kb = (
    "Common issues include login errors, app crashes, and network connectivity. "
    "To fix login errors, check your email and reset your password. "
    "For crashes, try reinstalling the app. If your connection is unstable, restart your router."
)

def tech_support_agent(state: State):
    prompt = (
        "You are a tech support assistant. Use the knowledge base below to answer the user's question.\n\n"
        f"Knowledge Base:\n{tech_support_kb}\n\n"
        f"User query:\n{state['input']}"
    )
    result = llm.invoke(prompt)
    return {"output": result.content}


# General Info Agent
general_info_kb = (
    "We offer 3 subscription plans: Basic, Pro, and Enterprise. "
    "Support is available 24/7. You can cancel your subscription any time from the account settings page."
)

def general_info_agent(state: State):
    prompt = (
        "You are a general info assistant. Use the knowledge base below to answer the user's question.\n\n"
        f"Knowledge Base:\n{general_info_kb}\n\n"
        f"User query:\n{state['input']}"
    )
    result = llm.invoke(prompt)
    return {"output": result.content}



# Intent Classification & Routing Logic
This section introduces the decision-making logic that powers the routing mechanism:

**classify_intent Node**: Uses an LLM wrapped with a structured output schema to classify the user's query into one of three support categories — billing, tech_support, or general_info. The result is stored in the state's decision key.

**route_to_agent Function**: A conditional router that examines the classification result and sends the request to the corresponding specialized agent node. This allows LangGraph to dynamically direct queries to the most relevant response module.

In [ ]:
def classify_intent(state: State):
    decision = router.invoke(
        [
            SystemMessage(content="Classify this support request as billing, tech_support, or general_info."),
            HumanMessage(content=state["input"]),
        ]
    )
    return {"decision": decision.step}

def route_to_agent(state: State):
    if state["decision"] == "billing":
        return "billing_agent"
    elif state["decision"] == "tech_support":
        return "tech_support_agent"
    elif state["decision"] == "general_info":
        return "general_info_agent"


# Building the Routing Graph
Here, we construct the complete LangGraph workflow by registering the nodes and defining their connections:

**Node Registration**: All agents (billing_agent, tech_support_agent, general_info_agent) and the classify_intent node are added to the graph.

**Conditional Routing**: After the graph starts at classify_intent, it uses the output of route_to_agent to forward the query to the appropriate agent node based on the classified intent.

**Terminal Edges**: Each agent node directly leads to END, finalizing the workflow after responding to the query.

This design reflects a typical customer support router architecture, enabling modular, extensible handling of diverse query types.

In [ ]:
builder = StateGraph(State)

builder.add_node("classify_intent", classify_intent)
builder.add_node("billing_agent", billing_agent)
builder.add_node("tech_support_agent", tech_support_agent)
builder.add_node("general_info_agent", general_info_agent)

builder.add_edge(START, "classify_intent")

builder.add_conditional_edges(
    "classify_intent",
    route_to_agent,
    {
        "billing_agent": "billing_agent",
        "tech_support_agent": "tech_support_agent",
        "general_info_agent": "general_info_agent",
    },
)

builder.add_edge("billing_agent", END)
builder.add_edge("tech_support_agent", END)
builder.add_edge("general_info_agent", END)

workflow = builder.compile()

# Let's run some queries:

In [ ]:
queries = [
    "Why was I charged $39.99 this month?",                    # billing
    "The app keeps crashing when I open it.",                  # tech support
    "Can I cancel my subscription anytime?",                   # general info
    "Can you show me all my past invoices?",                   # billing
    "How do I fix login issues with my account?"               # tech support
]

# Run each query through the router
for i, query in enumerate(queries, start=1):
    print(f"\n--- Query {i} ---")
    state = workflow.invoke({"input": query})
    print(f"Input: {query}")
    print(f"Response:\n{state['output']}")


--- Query 1 ---
Input: Why was I charged $39.99 this month?
Response:
You were charged $39.99 this month for Invoice INV003, which was issued on January 1, 2025. This amount reflects a change in your billing plan or an upgrade that occurred, as your previous invoices were for $29.99 each. If you need more details about the change in your plan or if you believe this charge is incorrect, please let me know!

--- Query 2 ---
Input: The app keeps crashing when I open it.
Response:
It sounds like you're experiencing an app crash issue. To resolve this, try reinstalling the app. This often fixes any underlying problems that may be causing the crashes. If the issue persists after reinstalling, please let me know!

--- Query 3 ---
Input: Can I cancel my subscription anytime?
Response:
Yes, you can cancel your subscription anytime from the account settings page.

--- Query 4 ---
Input: Can you show me all my past invoices?
Response:
Sure! Here are your past invoices:

1. **Invoice INV001**
   

# Make sure to view your traces in Phoenix!